<a href="https://colab.research.google.com/github/AI-Wardrobe/AI-WardrobeCode/blob/main/image_tagger.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install torch torchvision pillow pandas tqdm


In [ ]:
import os
from PIL import Image

IMAGE_DIR = "/content/drive/MyDrive/ai-closet"
image_paths = [os.path.join(IMAGE_DIR, f)
               for f in os.listdir(IMAGE_DIR)
               if f.lower().endswith(('.jpg','.jpeg','.png'))]

print("Number of images found:", len(image_paths))
image_paths[:5]

Number of images found: 26


['/content/drive/MyDrive/ai-closet/brownhat.jpg',
 '/content/drive/MyDrive/ai-closet/browncoat.jpeg',
 '/content/drive/MyDrive/ai-closet/blackshoes.jpeg',
 '/content/drive/MyDrive/ai-closet/colorfulshoes.jpeg',
 '/content/drive/MyDrive/ai-closet/blackcat4s.jpeg']

In [ ]:
import torch
from torchvision import models, transforms

model = models.resnet50(weights='ResNet50_Weights.DEFAULT')
model.eval()

transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
])


Downloading: "https://download.pytorch.org/models/resnet50-11ad3fa6.pth" to /root/.cache/torch/hub/checkpoints/resnet50-11ad3fa6.pth


100%|██████████| 97.8M/97.8M [00:00<00:00, 335MB/s]


In [ ]:
import torch.nn.functional as F

def extract_features(img_path):
    try:
        img = Image.open(img_path).convert("RGB")
    except:
        return None

    tensor = transform(img).unsqueeze(0)

    with torch.no_grad():
        outputs = model(tensor)
        probs = F.softmax(outputs, dim=1)

    top_prob, top_catid = torch.topk(probs, 5)

    return {
        "image": img_path,
        "top_classes": top_catid.cpu().numpy().tolist(),
        "top_scores": top_prob.cpu().numpy().tolist()
    }


In [ ]:
import pandas as pd
from tqdm import tqdm

rows = []

for path in tqdm(image_paths):
    result = extract_features(path)
    if result:
        rows.append(result)

df = pd.DataFrame(rows)
df.head()


100%|██████████| 26/26 [00:20<00:00,  1.24it/s]


,image,top_classes,top_scores
0,/content/drive/MyDrive/ai-closet/brownhat.jpg,"[[515, 439, 808, 765, 678]]","[[0.1971515268087387, 0.12173765152692795, 0.1..."
1,/content/drive/MyDrive/ai-closet/browncoat.jpeg,"[[841, 869, 911, 568, 636]]","[[0.12571650743484497, 0.07366778701543808, 0...."
2,/content/drive/MyDrive/ai-closet/blackshoes.jpeg,"[[630, 770, 502, 902, 797]]","[[0.15941189229488373, 0.08009650558233261, 0...."
3,/content/drive/MyDrive/ai-closet/colorfulshoes...,"[[502, 774, 630, 464, 788]]","[[0.3068508803844452, 0.10651867091655731, 0.0..."
4,/content/drive/MyDrive/ai-closet/blackcat4s.jpeg,"[[770, 502, 774, 630, 788]]","[[0.3507956862449646, 0.09852570295333862, 0.0..."


In [ ]:
!wget -q https://raw.githubusercontent.com/pytorch/hub/master/imagenet_classes.txt


In [ ]:
with open("imagenet_classes.txt") as f:
    classes = [line.strip() for line in f.readlines()]

df["labels"] = df["top_classes"].apply(lambda ids: [classes[i] for i in ids[0]])

df.head()


,image,top_classes,top_scores,labels
0,/content/drive/MyDrive/ai-closet/brownhat.jpg,"[[515, 439, 808, 765, 678]]","[[0.1971515268087387, 0.12173765152692795, 0.1...","[cowboy hat, bearskin, sombrero, rocking chair..."
1,/content/drive/MyDrive/ai-closet/browncoat.jpeg,"[[841, 869, 911, 568, 636]]","[[0.12571650743484497, 0.07366778701543808, 0....","[sweatshirt, trench coat, wool, fur coat, mail..."
2,/content/drive/MyDrive/ai-closet/blackshoes.jpeg,"[[630, 770, 502, 902, 797]]","[[0.15941189229488373, 0.08009650558233261, 0....","[Loafer, running shoe, clog, whistle, sleeping..."
3,/content/drive/MyDrive/ai-closet/colorfulshoes...,"[[502, 774, 630, 464, 788]]","[[0.3068508803844452, 0.10651867091655731, 0.0...","[clog, sandal, Loafer, buckle, shoe shop]"
4,/content/drive/MyDrive/ai-closet/blackcat4s.jpeg,"[[770, 502, 774, 630, 788]]","[[0.3507956862449646, 0.09852570295333862, 0.0...","[running shoe, clog, sandal, Loafer, shoe shop]"


In [ ]:
output_path = "/content/drive/MyDrive/ai-closet/tags.csv"
df.to_csv(output_path, index=False)
output_path


'/content/drive/MyDrive/ai-closet/tags.csv'

In [ ]:
print("Tagging complete! File saved to:", output_path)


Tagging complete! File saved to: /content/drive/MyDrive/ai-closet/tags.csv
